<a href="https://colab.research.google.com/github/MahmoudAlaa123/eT3-assessment-BigData/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Detection system using YOLO

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.5/609.5 kB 7.8 MB/s eta 0:00:00


In [3]:
import os
from ultralytics import YOLO
import json

In [4]:
ROOTDIR = '/content/gdrive/My Drive/et3_drive'

In [5]:
model = YOLO('yolov8n.pt')

results = model.train(data= os.path.join(ROOTDIR,"custom_data.yaml"),epochs = 100,save_txt=True)

100%|██████████| 6.23M/6.23M [00:00<00:00, 68.0MB/s]
Ultralytics YOLOv8.0.161 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/gdrive/My Drive/et3_drive/custom_data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=True, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

In [6]:
!yolo predict model='/content/runs/detect/train/weights/best.pt' conf=0.25 source='/content/gdrive/MyDrive/et3_drive/test/images/*.jpg'

# Convert the labels of test data to JSON

In [7]:
def process_text_file(input_file):
    annotations = []

    with open(input_file, "r") as file:
        lines = file.readlines()
        for line in lines:
            values = line.strip().split()
            x, y, width, height = map(float, values[1:])
            image_name = os.path.basename(input_file.replace(".txt", ".jpg"))
            image_url = f"/data/upload/{image_name}"

            annotation = {
                "image_rotation": 0,
                "value": {
                    "x": x * 100,
                    "y": y * 100,
                    "width": width * 100,
                    "height": height * 100,
                    "rotation": 0,
                    "rectanglelabels": ["object"]
                }
            }
            annotations.append(annotation)

    data = {
        "annotations": [
            {
                "result": annotations
            }
        ],
        "data": {
            "image": image_url
        }
    }

    return data



## Storing the output in drive

In [8]:
input_folder = "/content/gdrive/MyDrive/et3_drive/test/labels"
output_folder = "/content/gdrive/MyDrive/et3_drive/json_output"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename.replace(".txt", ".json"))

        data = process_text_file(input_file_path)

        with open(output_file_path, "w") as json_file:
            json.dump(data, json_file, indent=4)